Model Transformer

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import pandas as pd
import numpy as np


2023-03-17 10:30:52.202122: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 10:30:55.111872: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/tyler/miniconda3/envs/tf/lib/
2023-03-17 10:30:55.121059: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/tyler/miniconda3/envs/tf/lib/
2023-03-17 10:30:55.121095: W tensorflow/compiler/tf2ten

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf


def convertTextToNumbers(words, tokenizer=None, padding=None, isTrainY=False):
    # Create a tokenizer and fit it on the entire text corpus
    if tokenizer == None:
        #tokenizer = Tokenizer(split=' ')
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(words)

    # Convert the text to sequences of integers
    sequences = tokenizer.texts_to_sequences(words)

    if isTrainY:
        tokenizer.word_index['<start>'] = len(
            tokenizer.word_index) + 1
        tokenizer.word_index['<end>'] = len(
            tokenizer.word_index) + 1
        sequences = [[tokenizer.word_index['<start>']] +
                     seq + [tokenizer.word_index['<end>']] for seq in sequences]

    # Get the word index mapping
    word_index = tokenizer.word_index

    # Making sure all inputs are of the same length
    if padding == None:
        sequences = tf.keras.preprocessing.sequence.pad_sequences(
            sequences, padding='post')
    else:
        sequences = tf.keras.preprocessing.sequence.pad_sequences(
            sequences, maxlen=padding, padding='post')
    return (sequences, word_index, tokenizer)


In [3]:
import pickle
def saveTokenizers(train_x_tokenizer, train_y_tokenizer):
    with open('train_x_tokenizer.pickle', 'wb') as handle:
        pickle.dump(train_x_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open('train_y_tokenizer.pickle', 'wb') as handle:
        pickle.dump(train_y_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [4]:
import pickle
def loadTokenizers():

    with open('train_x_tokenizer.pickle', 'rb') as handle:
        train_x_tokenizer = pickle.load(handle)
    with open('train_y_tokenizer.pickle', 'rb') as handle:
        train_y_tokenizer = pickle.load(handle)

    return (train_x_tokenizer, train_y_tokenizer)


In [5]:
fullDs = pd.read_csv("fullDataset.csv")

In [6]:
fullDs = fullDs.sample(frac=1)  # shuffle the dataset
fullDs = fullDs.astype(str)

# the two arrays below are tokenized and padded for the algorithm
full_train_x, defWordIndex, defTokenizer = convertTextToNumbers(
    fullDs['definition'].values[0:10000])

full_train_y, simpDefWordIndex, simpDefTokenizer = convertTextToNumbers(
    fullDs['simplified_definition'].values[0:10000], isTrainY=True)


saveTokenizers(full_train_x, full_train_y)

full_train_y_RAW = fullDs['simplified_definition'].values[0:10000]


In [7]:
def data_generator(input_sequences, target_sequences, target_RAW, batch_size, simpWordIndex, lengthOfSentence):
    while True:
        for i in range(0, len(target_RAW), batch_size):

            decoder_target_data = np.zeros(
                (batch_size, lengthOfSentence, len(simpWordIndex)+1), dtype='float32')
            for index, sentence in enumerate(target_RAW[i:i+batch_size]):
                #print(sentence)
                tokens = sentence.split(' ')
                for j, token in enumerate(tokens):
                    temp = simpWordIndex.get(token, 0)
                    if temp > 0:

                        decoder_target_data[index, j, temp] = 1.0

            yield [input_sequences[i:i+batch_size], target_sequences[i:i+batch_size]], decoder_target_data


In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


# Just add the input_vocab_size and target_vocab_size
def createModel(input_vocab_size, target_vocab_size, num_layers=4, d_model=128, num_heads=8, dff=512, dropout_rate=.1):

    # Define the model hyperparameters
    #num_layers = 4
    #d_model = 128
    #num_heads = 8
    #dff = 512
    #input_vocab_size = 10000
    #target_vocab_size = 10000
    #dropout_rate = 0.1

    # Define the input and output sequences
    input_seq = layers.Input(shape=(None,))
    target_seq = layers.Input(shape=(None,))

    # Define the positional encoding layer
    positional_encoding = layers.Embedding(input_vocab_size, d_model)

    # Define the encoder layer
    encoder_inputs = positional_encoding(input_seq)
    encoder_outputs = encoder_inputs
    for i in range(num_layers):
        encoder_outputs = layers.MultiHeadAttention(
            num_heads, d_model)(encoder_outputs, encoder_outputs)
        encoder_outputs = layers.Dropout(dropout_rate)(encoder_outputs)
        encoder_outputs = layers.LayerNormalization(epsilon=1e-6)(encoder_outputs)
        encoder_outputs_ffn = layers.Dense(dff, activation='relu')(encoder_outputs)
        encoder_outputs_ffn = layers.Dense(d_model)(encoder_outputs_ffn)
        encoder_outputs = layers.Dropout(dropout_rate)(encoder_outputs_ffn)
        encoder_outputs = layers.LayerNormalization(epsilon=1e-6)(encoder_outputs)

    # Define the decoder layer
    decoder_inputs = positional_encoding(target_seq)
    decoder_outputs = decoder_inputs
    for i in range(num_layers):
        decoder_outputs = layers.MultiHeadAttention(
            num_heads, d_model)(decoder_outputs, decoder_outputs)
        decoder_outputs = layers.Dropout(dropout_rate)(decoder_outputs)
        decoder_outputs = layers.LayerNormalization(epsilon=1e-6)(decoder_outputs)
        decoder_outputs = layers.MultiHeadAttention(
            num_heads, d_model)(decoder_outputs, encoder_outputs)
        decoder_outputs = layers.Dropout(dropout_rate)(decoder_outputs)
        decoder_outputs = layers.LayerNormalization(epsilon=1e-6)(decoder_outputs)
        decoder_outputs_ffn = layers.Dense(dff, activation='relu')(decoder_outputs)
        decoder_outputs_ffn = layers.Dense(d_model)(decoder_outputs_ffn)
        decoder_outputs = layers.Dropout(dropout_rate)(decoder_outputs_ffn)
        decoder_outputs = layers.LayerNormalization(epsilon=1e-6)(decoder_outputs)

    # Define the output layer
    output_seq = layers.TimeDistributed(layers.Dense(target_vocab_size, activation='softmax')) (decoder_outputs)

    # Define the full transformer model
    transformer_model = keras.Model(
        inputs=[input_seq, target_seq], outputs=output_seq)

    # Define the loss function and optimizer
    #loss_fn = keras.losses.SparseCategoricalCrossentropy()
    loss_fn = keras.losses.CategoricalCrossentropy()
    optimizer = keras.optimizers.Adam()

    # Train the model
    transformer_model.compile(optimizer=optimizer, loss=loss_fn)

    return transformer_model



model = createModel(len(defWordIndex), len(simpDefWordIndex))
batch_size = 50

steps_per_epoch = len(full_train_x) // batch_size

generator = data_generator(full_train_x, full_train_y, full_train_y_RAW,
                           batch_size, simpDefWordIndex, len(full_train_y[0]))
model.fit(generator, epochs=5)


Epoch 1/5


2023-03-17 10:41:50.883468: W tensorflow/core/framework/op_kernel.cc:1818] INVALID_ARGUMENT: required broadcastable shapes


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/mul' defined at (most recent call last):
    File "/home/tyler/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/tyler/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/tyler/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/tyler/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/tyler/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/tyler/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/

In [ ]:
import numpy as np
data, tokens, tokenizer = convertTextToNumbers(
    ["West African ceremonial trumpet"], defTokenizer, padding=len(full_train_x[0]))
print(data)
print(defTokenizer.sequences_to_texts(data))

inputdata = np.array(data).reshape(1, -1)

outputdata = np.zeros(shape=(1, len(full_train_y[0])))
outputdata[0, 0] = simpDefTokenizer.word_index['<start>']


newSentence = model.predict([inputdata, outputdata])
print(newSentence)

In [ ]:
import numpy as np

def convertPredictionToSentence(ds, tokenizer):
    finalSentence = np.zeros((len(ds), len(ds[0])), dtype='int32')
    for i, sentence in enumerate(ds):
        for j, word in enumerate(sentence):
            maxIndex = np.argmax(word, axis=0)
            print(maxIndex)
            finalSentence[i,j] = maxIndex

    return tokenizer.sequences_to_texts(finalSentence)

In [ ]:
convertPredictionToSentence(newSentence, simpDefTokenizer)


In [6]:
defTokenizer, simpDefTokenizer = loadTokenizers()

In [15]:
defTokenizer.to_json()

'{"class_name": "Tokenizer", "config": {"num_words": null, "filters": "!\\"#$%&()*+,-./:;<=>?@[\\\\]^_`{|}~\\t\\n", "lower": true, "split": " ", "char_level": false, "oov_token": null, "document_count": 10000, "word_counts": "{\\"of\\": 9092, \\"woman\\": 48, \\"to\\": 9038, \\"make\\": 320, \\"visual\\": 1, \\"or\\": 8084, \\"visible\\": 23, \\"see\\": 385, \\"in\\": 3222, \\"fancy\\": 13, \\"the\\": 8960, \\"act\\": 423, \\"state\\": 344, \\"possessing\\": 21, \\"holding\\": 24, \\"as\\": 3544, \\"one\'s\\": 167, \\"own\\": 41, \\"that\\": 603, \\"department\\": 12, \\"sanitary\\": 6, \\"science\\": 68, \\"which\\": 1146, \\"treats\\": 17, \\"preservation\\": 13, \\"health\\": 27, \\"esp\\": 159, \\"households\\": 2, \\"and\\": 2573, \\"communities\\": 2, \\"a\\": 8953, \\"system\\": 59, \\"principles\\": 26, \\"rules\\": 27, \\"designated\\": 9, \\"for\\": 1097, \\"promotion\\": 3, \\"character\\": 86, \\"part\\": 261, \\"play\\": 39, \\"specific\\": 19, \\"kind\\": 140, \\"manifest